*   Make sure you run this using a GPU (On Google Colab: Runtime -> Change Runtime Type -> GPU)

# Setup

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
!pip install -q ase
!pip install -q torch==1.8.0
!pip install  torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git@1.7.0

     |████████████████████████████████| 2.2MB 4.1MB/s 
     |████████████████████████████████| 735.5MB 23kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
     |████████████████████████████████| 2.7MB 2.1MB/s 
     |████████████████████████████████| 1.6MB 5.2MB/s 
     |████████████████████████████████| 1.1MB 3.9MB/s 
     |████████████████████████████████| 399kB 4.1MB/s 
     |████████████████████████████████| 235kB 4.1MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 


In [ ]:
import numpy as np
import torch
import ase
import random
from torch_geometric.data import Data, DataLoader
from ase.io import read

In [ ]:
SEED = 55555

np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

## Set training hyperparameters

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # use GPU if available
DTYPE = torch.float64  # data type to use for data and model
TRAINING_RATIO = 0.8  # percent of the dataset to use for training
OPTIMIZER = torch.optim.Adam  
BATCH_SIZE = 6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/GNN_atomistics/'  # path to the directory where the git repo is stored
DATA_PATH = BASE_PATH + 'data/training'
MODELS_PATH = BASE_PATH + "models"

Mounted at /content/drive


# Load data

## Function definitions

In [ ]:
M = {}
def extend_atoms(atoms, source, target):
  global M
  if source not in M.keys():
    M[source] = {}
  if target not in M[source].keys():
    M[source][target] = ase.build.find_optimal_cell_shape(atoms.get_cell(), target, "sc") 
  supercell = ase.build.make_supercell(atoms, M[source][target])
  supercell.info["energy"] = atoms.info["energy"] * int(target / source)
  return supercell

In [ ]:
def data_object(atoms: ase.Atoms):

  n = atoms.get_global_number_of_atoms()
  if n == 1:
    atoms = extend_atoms(atoms, 1, 54)
    n = atoms.get_global_number_of_atoms()

  cell = torch.tensor(atoms.cell, dtype=DTYPE, device=DEVICE)
  x = torch.tensor(atoms.get_positions(), dtype=DTYPE, device=DEVICE)
  z = torch.tensor(atoms.get_array("numbers", copy=True), dtype=torch.long, device=DEVICE)
  y = torch.tensor(atoms.info["energy"], dtype=DTYPE, device=DEVICE)
  f = torch.tensor(atoms.get_array("force", copy=True), dtype=DTYPE, device=DEVICE)

  return Data(z=z, x=x, cell=cell, y=y, f=f, n=n)

## Load data

In [ ]:
data = []
file_names = [f"DB{i}.xyz" for i in range(1,9)]
# file_names = ["DB1_100.xyz"]  # only 100 samples from DB1; for debugging
file_paths = [f"{DATA_PATH}/{f}" for f in file_names]
for f in file_paths:
  db = read(f, index=":")
  data += list(map(data_object, db))
print(len(data))

12171


In [ ]:
y_atom = torch.tensor([d.y / d.n for d in data], dtype=DTYPE)
y_atom_mean = y_atom.mean().item()
y_atom_std = y_atom.std().item()
print(y_atom_mean, y_atom_std)

-3460.825847482401 0.16576383029449515


In [ ]:
random.shuffle(data)
train_amount = int(len(data) * TRAINING_RATIO)
train_data = data[:train_amount]
test_data = data[train_amount:]
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# Model definitions

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list 

def pbc_edges(cutoff, z, x, cell, batch, compute_sc=False):

  NH1 = torch.tensor([], dtype=torch.long, device=DEVICE)
  NH2 = torch.tensor([], dtype=torch.long, device=DEVICE)
  S = torch.tensor([], dtype=torch.long, device=DEVICE)
  D = torch.tensor([], dtype=DTYPE, device=DEVICE)
  SC = torch.tensor([], dtype=DTYPE, device=DEVICE) if compute_sc else None
  x_ = torch.clone(x).detach().cpu().numpy()

  if batch is not None:
    # count number of elements per batch
    batch_ids = list(set(batch.cpu().tolist()))
    batch_sizes = [ (batch == id).sum().item() for id in batch_ids ]

    for i in range(len(batch_sizes)):
      offset = sum(batch_sizes[:i]) # to obtain correct atom indices
      
      atoms = Atoms(charges = (z[offset:offset + batch_sizes[i]]).cpu(), 
        positions = x_[offset:offset + batch_sizes[i]], 
        cell = (cell[3*i:3*(i+1)]).cpu(),
        pbc = True
      ) 
      
      nh1, nh2, s = neighbor_list("ijS", atoms, cutoff, self_interaction=False) 
      nh1 = torch.tensor(nh1, dtype=torch.long, device=DEVICE)
      nh2 = torch.tensor(nh2, dtype=torch.long, device=DEVICE)
      nh1 = nh1 + offset
      nh2 = nh2 + offset
      s = torch.tensor(s, dtype=DTYPE, device=DEVICE)
      d = x[nh2] - x[nh1] + torch.matmul(s, cell[3*i:3*(i+1)])
      
      if compute_sc:
        cell_flat = torch.flatten(cell[3*i:3*(i+1)])
        sc = torch.tile(cell_flat, (len(d), 1))
        sc[:, 0:3] = (sc[:, 0:3].T * s[:, 0]).T
        sc[:, 3:6] = (sc[:, 3:6].T * s[:, 1]).T
        sc[:, 6:9] = (sc[:, 6:9].T * s[:, 2]).T
        SC = torch.cat((SC, sc), 0)      

      NH1 = torch.cat((NH1, nh1), 0)
      NH2 = torch.cat((NH2, nh2), 0)
      S = torch.cat((S, s), 0)
      D = torch.cat((D, d), 0)

  else: # no batch
    atoms = Atoms(charges = z.cpu(), positions = x.cpu(), cell = cell.cpu(), pbc = True)
    nh1, nh2, s = neighbor_list("ijS", atoms, cutoff, self_interaction=False)
    nh1 = torch.tensor(nh1, dtype=torch.long, device=DEVICE)
    nh2 = torch.tensor(nh2, dtype=torch.long, device=DEVICE)
    s = torch.tensor(s, dtype=DTYPE, device=DEVICE)
    d = x[nh2] - x[nh1] + torch.matmul(s, cell)
    
    if compute_sc:
      cell_flat = torch.flatten(cell)
      sc = torch.tile(cell_flat, (len(d), 1))
      sc[:, 0:3] = (sc[:, 0:3].T * s[:, 0]).T
      sc[:, 3:6] = (sc[:, 3:6].T * s[:, 1]).T
      sc[:, 6:9] = (sc[:, 6:9].T * s[:, 2]).T  
      SC = sc

    NH1, NH2, S, D = nh1, nh2, s, d

  D = D.norm(dim=-1)
  return  NH1, NH2, D, S, SC 

## Schnet

In [ ]:
import os
import warnings
import os.path as ospa
from math import pi as PI
import torch.nn.functional as F
from torch.nn import Embedding, Sequential, Linear, ModuleList
from torch_scatter import scatter
from torch_geometric.data.makedirs import makedirs
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn import radius_graph, MessagePassing

class SchNet2(torch.nn.Module):

    def __init__(self, hidden_channels=128, num_filters=128,
                 num_interactions=6, num_gaussians=50, cutoff=10.0,
                 readout='add', dipole=False, mean=None, std=None,
                 atomref=None):
        super(SchNetx, self).__init__()

        assert readout in ['add', 'sum', 'mean']

        self.hidden_channels = hidden_channels
        self.num_filters = num_filters
        self.num_interactions = num_interactions
        self.num_gaussians = num_gaussians
        self.cutoff = cutoff
        self.readout = readout
        self.dipole = dipole
        self.readout = 'add' if self.dipole else self.readout
        self.mean = mean
        self.std = std
        self.scale = None

        atomic_mass = torch.from_numpy(ase.data.atomic_masses)
        self.register_buffer('atomic_mass', atomic_mass)

        self.embedding = Embedding(100, hidden_channels)
        self.distance_expansion = GaussianSmearing(0.0, cutoff, num_gaussians)

        self.interactions = ModuleList()
        for _ in range(num_interactions):
            block = InteractionBlock(hidden_channels, num_gaussians,
                                     num_filters, cutoff)
            self.interactions.append(block)

        self.lin1 = Linear(hidden_channels, hidden_channels // 2)
        self.act = ShiftedSoftplus()
        self.lin2 = Linear(hidden_channels // 2, 1)

        self.register_buffer('initial_atomref', atomref)
        self.atomref = None
        if atomref is not None:
            self.atomref = Embedding(100, 1)
            self.atomref.weight.data.copy_(atomref)
        
        self.reset_parameters()

    def reset_parameters(self):
        self.embedding.reset_parameters()
        for interaction in self.interactions:
            interaction.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        self.lin1.bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)
        if self.atomref is not None:
            self.atomref.weight.data.copy_(self.initial_atomref)

    def forward(self, z, x, cell=None, batch=None):
        assert z.dim() == 1 and z.dtype == torch.long
        batch = torch.zeros_like(z) if batch is None else batch

        h = self.embedding(z)

        if cell != None:
          row, col, edge_weight, shift, _ = pbc_edges(self.cutoff, z, x, cell, batch, compute_sc=False)
          edge_index = torch.stack((row, col))
        else:
          edge_index = radius_graph(x, r=self.cutoff, batch=batch)
          row, col = edge_index
          edge_weight = (x[row] - x[col]).norm(dim=-1)
        
        edge_attr = self.distance_expansion(edge_weight)

        for interaction in self.interactions:
            h = h + interaction(h, edge_index, edge_weight, edge_attr)

        h = self.lin1(h)
        h = self.act(h)
        h = self.lin2(h)

        if self.dipole:
            mass = self.atomic_mass[z].view(-1, 1)
            c = scatter(mass * x, batch, dim=0) / scatter(mass, batch, dim=0)
            h = h * (pos - c[batch])

        if not self.dipole and self.mean is not None and self.std is not None:
            h = h * self.std + self.mean

        if not self.dipole and self.atomref is not None:
            h = h + self.atomref(z)

        out = scatter(h, batch, dim=0, reduce=self.readout)
        
        if self.dipole:
            out = torch.norm(out, dim=-1, keepdim=True)

        if self.scale is not None:
            out = self.scale * out

        return out


    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'hidden_channels={self.hidden_channels}, '
                f'num_filters={self.num_filters}, '
                f'num_interactions={self.num_interactions}, '
                f'num_gaussians={self.num_gaussians}, '
                f'cutoff={self.cutoff})')



class InteractionBlock(torch.nn.Module):
    def __init__(self, hidden_channels, num_gaussians, num_filters, cutoff):
        super(InteractionBlock, self).__init__()
        self.mlp = Sequential(
            Linear(num_gaussians, num_filters),
            ShiftedSoftplus(),
            Linear(num_filters, num_filters),
        )
        self.conv = CFConv(hidden_channels, hidden_channels, num_filters,
                           self.mlp, cutoff)
        self.act = ShiftedSoftplus()
        self.lin = Linear(hidden_channels, hidden_channels)

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[0].bias.data.fill_(0)
        self.conv.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin.weight)
        self.lin.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        x = self.conv(x, edge_index, edge_weight, edge_attr)
        x = self.act(x)
        x = self.lin(x)
        return x


class CFConv(MessagePassing):
    def __init__(self, in_channels, out_channels, num_filters, nn, cutoff):
        super(CFConv, self).__init__(aggr='add')
        self.lin1 = Linear(in_channels, num_filters, bias=False)
        self.lin2 = Linear(num_filters, out_channels)
        self.nn = nn
        self.cutoff = cutoff

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        C = 0.5 * (torch.cos(edge_weight * PI / self.cutoff) + 1.0)
        W = self.nn(edge_attr) * C.view(-1, 1)

        x = self.lin1(x)
        x = self.propagate(edge_index, x=x, W=W)
        x = self.lin2(x)
        return x

    def message(self, x_j, W):
        return x_j * W


class GaussianSmearing(torch.nn.Module):
    def __init__(self, start=0.0, stop=5.0, num_gaussians=50):
        super(GaussianSmearing, self).__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = (-0.5 / (offset[1] - offset[0]).item()**2)
        self.register_buffer('offset', offset)

    def forward(self, dist):
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        res = torch.exp(self.coeff * torch.pow(dist, 2))
        return res

class ShiftedSoftplus(torch.nn.Module):
    def __init__(self):
        super(ShiftedSoftplus, self).__init__()
        self.shift = torch.log(torch.tensor(2.0)).item()

    def forward(self, x):
        return F.softplus(x) - self.shift

## Dimenet

In [ ]:
#@title  { vertical-output: true, form-width: "25%" }
from torch_geometric.nn import DimeNet
from torch_geometric.nn.acts import swish
from math import sqrt, pi as PI

import numpy as np
import torch
from torch.nn import Linear, Embedding
from torch_scatter import scatter
from torch_sparse import SparseTensor
from torch_geometric.nn import radius_graph
from torch_geometric.data import download_url
from torch_geometric.data.makedirs import makedirs

from torch_geometric.nn.models.dimenet import Envelope
from torch_geometric.nn.models.dimenet import BesselBasisLayer
from torch_geometric.nn.models.dimenet import SphericalBasisLayer
from torch_geometric.nn.models.dimenet import ResidualLayer
from torch_geometric.nn.models.dimenet import InteractionBlock
from torch_geometric.nn.models.dimenet import OutputBlock

from torch_geometric.nn.models.dimenet_utils import bessel_basis, real_sph_harm
import ase
from ase.neighborlist import neighbor_list 
from ase import Atoms

from torch_geometric.data import DataLoader

try:
    import sympy as sym
except ImportError:
    sym = None

import os
try:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    import tensorflow as tf
except ImportError:
    tf = None

class EmbeddingBlock(torch.nn.Module):
  def __init__(self, num_radial, hidden_channels, act=swish):
    super(EmbeddingBlock, self).__init__()
    self.act = act

    self.emb = Embedding(95, hidden_channels)
    self.lin_rbf = Linear(num_radial, hidden_channels, bias=False)
    self.lin = Linear(3 * hidden_channels, hidden_channels)

    self.reset_parameters()

  def reset_parameters(self):
    self.emb.weight.data.uniform_(-sqrt(3), sqrt(3))
    self.lin_rbf.reset_parameters()
    self.lin.reset_parameters()

  def forward(self, x, rbf, i, j):
    x = self.emb(x)
    #rbf = self.act(self.lin_rbf(rbf)) # FIX: this should not have an activation function
    rbf = self.lin_rbf(rbf)
    return self.act(self.lin(torch.cat([x[i], x[j], rbf], dim=-1)))



class DimeNet2(torch.nn.Module):
  
  def __init__(self, hidden_channels, out_channels, num_blocks, num_bilinear,
                 num_spherical, num_radial, cutoff=5.0, envelope_exponent=5,
                 num_before_skip=1, num_after_skip=2, num_output_layers=3,
                 act=swish, mean=None, std=None):
    super(DimeNetx, self).__init__()

    self.cutoff = cutoff

    #set mean and standard deviation of energies
    self.mean = mean 
    self.std = std

    # padding used for PBCs
    self.padding = torch.nn.ConstantPad2d((0,6,0,0), 0)

    if sym is None:
        raise ImportError('Package `sympy` could not be found.')

    self.num_blocks = num_blocks

    self.rbf = BesselBasisLayer(num_radial, cutoff, envelope_exponent)
    self.sbf = SphericalBasisLayer(num_spherical, num_radial, cutoff,
                                    envelope_exponent)

    self.emb = EmbeddingBlock(num_radial, hidden_channels, act)

    self.output_blocks = torch.nn.ModuleList([
        OutputBlock(num_radial, hidden_channels, out_channels,
                    num_output_layers, act) for _ in range(num_blocks + 1)
    ])

    self.interaction_blocks = torch.nn.ModuleList([
        InteractionBlock(hidden_channels, num_bilinear, num_spherical,
                          num_radial, num_before_skip, num_after_skip, act)
        for _ in range(num_blocks)
    ])

    self.reset_parameters()

  def reset_parameters(self):
    self.rbf.reset_parameters()
    self.emb.reset_parameters()
    for out in self.output_blocks:
      out.reset_parameters()
    for interaction in self.interaction_blocks:
      interaction.reset_parameters()

  def triplets_original(self, edge_index, num_nodes):
    row, col = edge_index  # j->i

    value = torch.arange(row.size(0), device=row.device)
    adj_t = SparseTensor(row=col, col=row, value=value,
                         sparse_sizes=(num_nodes, num_nodes))
    adj_t_row = adj_t[row]
    num_triplets = adj_t_row.set_value(None).sum(dim=1).to(torch.long)

    # Node indices (k->j->i) for triplets.
    idx_i = col.repeat_interleave(num_triplets)
    idx_j = row.repeat_interleave(num_triplets)
    idx_k = adj_t_row.storage.col()
    mask = (idx_i != idx_k)  # Remove i == k triplets.
    idx_i, idx_j, idx_k = idx_i[mask], idx_j[mask], idx_k[mask]

    # Edge indices (k-j, j->i) for triplets.
    idx_kj = adj_t_row.storage.value()[mask]
    idx_ji = adj_t_row.storage.row()[mask]

    return col, row, idx_i, idx_j, idx_k, idx_kj, idx_ji


  def triplets(self, edge_index, num_nodes, shift_cells=None, shift=None):
    row, col = edge_index  # j->i

    value = torch.arange(row.size(0), device=row.device)
    adj_t = SparseTensor(row=col, col=row, value=value,
                         sparse_sizes=(num_nodes, num_nodes))
    adj_t_row = adj_t[row]
    num_triplets = adj_t_row.set_value(None).sum(dim=1).to(torch.long)

    idx_i = col.repeat_interleave(num_triplets)
    idx_j = row.repeat_interleave(num_triplets)
    idx_k = adj_t_row.storage.col()

    if shift_cells is not None: # Update also the shift vectors
      shift_cells_i = shift_cells.repeat_interleave(num_triplets, dim=0)
      shift_i = shift.repeat_interleave(num_triplets, dim=0)
      shift_cells_k = -shift_cells[adj_t_row.storage.value()]
      shift_k = -shift[adj_t_row.storage.value()]

    mask = torch.all((torch.cat((torch.unsqueeze(idx_i, 1), shift_i), dim=1) ==\
                      torch.cat((torch.unsqueeze(idx_k, 1), shift_k), dim=1)), dim=1)

    idx_i, idx_j, idx_k = idx_i[~mask], idx_j[~mask], idx_k[~mask]
    if shift_cells is not None: # Remove also from the shift vector
      shift_cells_i = shift_cells_i[~mask]
      shift_cells_k = shift_cells_k[~mask]
      shift_i = shift_i[~mask]
      shift_k = shift_k[~mask]

    idx_kj = adj_t_row.storage.value()[~mask]
    idx_ji = adj_t_row.storage.row()[~mask]

    return col, row, idx_i, idx_j, idx_k, idx_kj, idx_ji, shift_cells_i, shift_i, shift_cells_k, shift_k


  def forward(self, z, pos, cell=None, batch=None):

    edge_index = []
    dist = []
    shift_cells = None      
    
    edge_index = radius_graph(pos, r=self.cutoff, batch=batch)
    i, j, idx_i, idx_j, idx_k, idx_kj, idx_ji = self.triplets_original(
      edge_index, num_nodes=z.size(0))        
    dist = (pos[i] - pos[j]).pow(2).sum(dim=-1).sqrt()

    # Define atoms position 
    pos_i = pos[idx_i]
    pos_j = pos[idx_j]
    pos_k = pos[idx_k]

    # Calculate angles - with some Fixes to indexes compared to the orig. version
    pos_ji, pos_kj = pos_j - pos_i, pos_k - pos_j
    a = (pos_ji * pos_kj).sum(dim=-1)
    b = torch.cross(pos_ji, pos_kj)
    b = torch.linalg.norm(b + 1e-16, dim=-1)
    angle = torch.atan2(b, a)      

    rbf = self.rbf(dist)
    sbf = self.sbf(dist, angle, idx_kj)

    # Embedding block.
    x = self.emb(z, rbf, i, j)
    P = self.output_blocks[0](x, rbf, i, num_nodes=pos.size(0))

    # Interaction blocks.
    for interaction_block, output_block in zip(self.interaction_blocks,
                                               self.output_blocks[1:]):
      x = interaction_block(x, rbf, sbf, idx_kj, idx_ji)
      a = output_block(x, rbf, i, num_nodes=pos.size(0))
      P += a

    # Energy de-standardization
    if self.std is not None and self.mean is not None:
      P = P * self.std + self.mean
    
    
    return P, (P.sum(dim=0) if batch is None else scatter(P, batch, dim=0))

# Training

## Function definitions


In [ ]:
def energy_loss(data, p_energies):
  energies_loss = torch.mean(torch.abs(data.y - p_energies))
  return energies_loss

In [ ]:
def energy_forces_loss(data, p_energies, p_forces, energy_coeff):
  energies_loss = torch.mean(torch.abs(data.y - p_energies))
  forces_loss = torch.mean(torch.abs(data.f - p_forces))
  total_loss = (energy_coeff)*energies_loss + (1-energy_coeff)*forces_loss
  return total_loss, energies_loss, forces_loss

In [ ]:
def train(model, loader, optimizer, use_forces=False, energy_coeff=None):
  if not use_forces:
    train_energy(model, loader, optimizer)
  else:
    train_energy_forces(model, loader, optimizer, energy_coeff)

In [ ]:
def train_energy(model, loader, optimizer):
  model.train()
  for data in loader:
    optimizer.zero_grad()

    e = model(data.z, data.x, data.cell, data.batch)
    e = e.squeeze(1) 
    e_loss = energy_loss(data, e)

    e_loss.backward()
    optimizer.step()

In [ ]:
def train_energy_forces(model, loader, optimizer, energy_coeff):
  model.train()
  total_e_loss = 0
  total_f_loss = 0
  total_ef_loss = 0

  for data in loader:
    data.x.requires_grad = True 
    optimizer.zero_grad()
    
    e = model(data.z, data.x, data.cell, data.batch)
    f = -1 * torch.autograd.grad(e, data.x, grad_outputs=torch.ones_like(e), create_graph=True, retain_graph=True)[0]
    e = e.squeeze(1) 
    
    ef_loss, e_loss, f_loss = energy_forces_loss(data, e, f, energy_coeff)
    with torch.no_grad():
      total_e_loss += e_loss.item()
      total_f_loss += f_loss.item()
      total_ef_loss += ef_loss.item()

    ef_loss.backward()
    optimizer.step()

  print("Total training loss\t ef: {}, e: {}, f: {}".format(total_ef_loss, total_e_loss, total_f_loss))

In [ ]:
def test(model, train_loader, test_loader):
  model.eval()

  with torch.no_grad():
    train_errs = torch.tensor([], dtype=DTYPE)
    test_errs = torch.tensor([], dtype=DTYPE)
  
    for data in train_loader:  # Iterate in batches over the training/test dataset.
      e = model(data.z, data.x, data.cell, data.batch)
      e = e.squeeze(1)
      errs = torch.abs(e.view(-1).cpu() - data.y.cpu())
      train_errs = torch.cat((train_errs, errs))
    train_mae = torch.mean(train_errs).item()

    for data in test_loader:  # Iterate in batches over the training/test dataset.
      e = model(data.z, data.x, data.cell, data.batch)
      e = e.squeeze(1)
      errs = torch.abs(e.view(-1).cpu() - data.y.cpu())
      test_errs = torch.cat((test_errs, errs))
    test_mae = torch.mean(test_errs).item()

  return train_mae, test_mae

In [ ]:
from datetime import datetime

train_maes = []
test_maes = []
def train_and_test(model, train_loader, test_loader, optimizer, scheduler, 
                   use_forces=False, energy_coeff=None, 
                   epochs=100, starting_epoch=1, save_every=5,
                   early_stopping_starting_epoch=100, patience=100,
                   name=None, description=None):
  if name == None:
    name = "SchNet"
  if description == None:
    description = datetime.now().strftime('%H:%M:%S')

  lowest_test_mae = 100.
  epochs_not_improved = 0
  save_model_flag = True
  for epoch in range(starting_epoch, epochs):
    print("")
    print(f"Epoch {epoch} ({datetime.now().strftime('%H:%M:%S')})")
    print(f"")
    train(model, train_loader, optimizer, use_forces=use_forces, energy_coeff=energy_coeff)
    train_mae, test_mae = test(model, train_loader, test_loader)
    train_maes.append(train_mae)
    test_maes.append(test_mae)
    print(f"Energy MAE: train {train_mae} eV, test {test_mae} eV")
    if epoch <= early_stopping_starting_epoch:
      lowest_test_mae = test_mae
      save_model_flag = True
    else:
      print('Early stopping active')
      if test_mae < lowest_test_mae:
        epochs_not_improved = 0
        lowest_test_mae = test_mae
        save_model_flag = True
        print('MAE has decreased, saving the model')
      else:
        epochs_not_improved += 1
        save_model_flag = False
        if epochs_not_improved > patience:
          print('Patience has been finished: end of the training')
          break
        else:
          print('MAE has not decreased for ' + str(epochs_not_improved) + ' epochs, go on without saving the model')
    if save_model_flag == True:
      save_model(name=name, description=description, epoch=epoch, train_err=train_mae, test_err=test_mae)
    scheduler.step(test_mae)

In [ ]:
def save_model(name, description, epoch=-1, train_err=-1, test_err=-1):
  model_data = {
    "desc": "DimeNet new training should be the same as 12Apr",
    "str": model_str,
    "mean": y_atom_mean,
    "std": y_atom_std,
    "state": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scheduler": scheduler.state_dict(),
    "train_err": train_err,
    "test_err": test_err
  }  
  if not epoch == -1:
    epoch_str = "_" + str(epoch)
  else:
    epoch_str = ""
  actual_name = f"{name}{epoch_str}"

  dir = f"{MODELS_PATH}/{name}"
  if not os.path.exists(dir):
    os.makedirs(dir)
    print(f"Created directory {dir}")
  torch.save(model_data, f"{dir}/{actual_name}")
  print(f"Saved {actual_name}")

  if os.path.isfile(f"{dir}/{name}_best"):
    best = torch.load(f"{dir}/{name}_best")
    best_train_err = best["train_err"]
    best_test_err = best["test_err"]
  else:
    best_train_err, best_test_err = 10e18, 10e18
  # if train_err < best_train_err and test_err < best_test_err:
  if test_err < best_test_err:
    torch.save(model_data, f"{dir}/{name}_best")
    print(f"Saved best")

## Instantiate model

### Dimenet

In [ ]:
cutoff = 3.5
model = DimeNet2(hidden_channels=128, out_channels=1, num_blocks=7, num_bilinear=8, num_spherical=7, num_radial=6, cutoff=cutoff, std=y_atom_std, mean=y_atom_mean).to(device)

if DTYPE == torch.float64:
  model = model.double()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer.zero_grad()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='abs')
use_forces = False
energy_coeff = 0.
max_epochs = 100

### Schnet

In [ ]:
cutoff = 5.0
model = SchNet2(hidden_channels = 128, num_filters=128, num_gaussians=128, cutoff=cutoff, 
                num_interactions=3, readout="sum", mean=y_atom_mean, std=y_atom_std).to(DEVICE)
if DTYPE == torch.float64:
  model = model.double()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
optimizer.zero_grad()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, threshold=0.05, threshold_mode='rel')
use_forces = False
energy_coeff = 0.
max_epochs = 100

### Rest

In [ ]:
model_name = "" # name of the folder where checkpoints will be saved
model_description = "" # description to understand what model is what

Copy-paste the content of the cell used to instantiate the model in the following variable to allow for saving of hyperparameters in the checkpoint

In [ ]:
model_str = """
"""

# [Restore from checkpoint]

In [ ]:
RESTORE_FROM_CHECKPOINT = False

if RESTORE_FROM_CHECKPOINT:
  model_fn = "path/to/checkpoint_to_restore"
  checkpoint = torch.load("path/to/checkpoint", map_location=DEVICE)
  model.load_state_dict(checkpoint["state"])
  optimizer.load_state_dict(checkpoint["optimizer"])
  scheduler.load_state_dict(checkpoint["scheduler"])

# Info

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
def experiment_summary():
  device = "CPU" if DEVICE == torch.device("cpu") else f"GPU ({torch.cuda.get_device_name(device=DEVICE)})"
  print(f"Device: {DEVICE}")
  print(f"Data type: {DTYPE}")
  print(f"Cutoff: {cutoff}")
  print(f"Mean: {y_atom_mean}, std: {y_atom_std}")
  print(f"Training ratio: {TRAINING_RATIO}")
  print(f"Batch size: {BATCH_SIZE}")
  print(f"Model: \n    {model_name}: {model_description}\n    {str(model)}")
  loss = "energy" if not use_forces else f"energy+forces ({energy_coeff})"
  print(f"Loss: {loss}")
  print(f"Optimizer: {optimizer}")
  print(f"Scheduler: {scheduler}")
  print(f"Max epochs: {max_epochs}")
experiment_summary()

Device: GPU (Tesla P100-PCIE-16GB)
Data type: torch.float64
Files used: ['DB1.xyz', 'DB2.xyz', 'DB3.xyz', 'DB4.xyz', 'DB5.xyz', 'DB6.xyz', 'DB7.xyz', 'DB8.xyz']
Cutoff: 3.5
Mean: -3460.825847482401, std: 0.16576383029449515
Training ratio: 0.8
Batch size: 6
Model: 
    dimenet_no_PBC: DimeNet no PBC for LAMMPS testing
    DimeNetx(
  (padding): ConstantPad2d(padding=(0, 6, 0, 0), value=0)
  (rbf): BesselBasisLayer(
    (envelope): Envelope()
  )
  (sbf): SphericalBasisLayer(
    (envelope): Envelope()
  )
  (emb): EmbeddingBlock(
    (emb): Embedding(95, 128)
    (lin_rbf): Linear(in_features=6, out_features=128, bias=False)
    (lin): Linear(in_features=384, out_features=128, bias=True)
  )
  (output_blocks): ModuleList(
    (0): OutputBlock(
      (lin_rbf): Linear(in_features=6, out_features=128, bias=False)
      (lins): ModuleList(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): Linear(in_features=128, out_features=128, bias=True)
        (2): Linear

In [ ]:
#@title  { vertical-output: true, display-mode: "both" }
from psutil import *
print(cpu_count())
print(cpu_stats())
!cat /proc/cpuinfo
!df -h
print(virtual_memory())


4
scpustats(ctx_switches=4921552, interrupts=2873079, soft_interrupts=1748261, syscalls=0)
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogom

# Training

In [ ]:
train_and_test(model, train_loader, test_loader, optimizer, scheduler, 
               use_forces=use_forces, energy_coeff=energy_coeff,
               epochs=max_epochs, starting_epoch=1, save_every=1,
               early_stopping_starting_epoch=40, patience=10,
               name=model_name, description=model_description)